In [ ]:
import json
import csv

In [ ]:
input_file = 'data/dengue_data.txt'
output_file = 'data/google_dengue.json'

In [ ]:
country_names = []
with open("data/world-110m-country-names.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    for row in rd:
#         print(row[1])
        country_names.append(row[1])
country_names = set(country_names)

In [ ]:
country_names

In [ ]:
data_names = []
with open(input_file) as fd:
    rd = csv.reader(fd, delimiter=",")
    for row in rd:
        if row[0] == 'Date':
            data_names = row
            data_names.remove('Date')
            data_names = set(data_names)
            break

In [ ]:
data_names

In [ ]:
def string_convert(input_string):
    if input_string == 'Bolivia':
        return 'Bolivia, Plurinational State of'
    if input_string == 'Venezuela':
        return 'Venezuela, Bolivarian Republic of'
    else:
        return input_string

In [ ]:
for data_name in data_names:
    if string_convert(data_name) not in country_names:
        print(data_name)

In [ ]:
country_ids = {}
with open("data/world-110m-country-names.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    for row in rd:
        if row[1] == 'name':
            continue
        country_id = int(row[0])
        country_name = row[1]
        country_ids[country_name] = country_id

In [ ]:
dictionary = {}
country_mapping = {}
with open(input_file) as fd:
    rd = csv.reader(fd, delimiter=",")
    week_num = 1
    prev_year = 2002
    for row in rd:
        if row[0] == 'Date':
            dictionary['countries'] = {}
            for i in range(1,len(row)):
                if string_convert(row[i]) not in dictionary['countries']:
                    country_mapping[i] = string_convert(row[i])
                    dictionary['countries'][string_convert(row[i])] = {}
                    dictionary['countries'][string_convert(row[i])]['name'] = string_convert(row[i])
                    dictionary['countries'][string_convert(row[i])]['weeks'] = {}
                    if string_convert(row[i]) in country_ids:
                        dictionary['countries'][string_convert(row[i])]['id'] = country_ids[string_convert(row[i])]
                    else:
                        dictionary['countries'][string_convert(row[i])]['id'] = -9999
        else:
            if int(row[0].split('-')[0]) != prev_year:
                week_num = 1
                prev_year = int(row[0].split('-')[0])
            
            attribute = 'weekly dengue activity estimates for year ' + str(prev_year)
            
            for i in range(1,len(row)):
                if week_num not in dictionary['countries'][country_mapping[i]]['weeks']:
                    dictionary['countries'][country_mapping[i]]['weeks'][week_num] = {}
                if prev_year not in dictionary['countries'][country_mapping[i]]['weeks'][week_num]:
                    if row[i] == '':
                        dictionary['countries'][country_mapping[i]]['weeks'][week_num][attribute] = 0.0
                    else:
                        dictionary['countries'][country_mapping[i]]['weeks'][week_num][attribute] = float(row[i])
                    
            week_num += 1

In [ ]:
dictionary

In [ ]:
with open(output_file, 'w') as fp:
    json.dump(dictionary, fp)